In [ ]:
import sqlite3
import pandas as pd

# 数据库路径
db_path = r'sgmkg.db'

conn = sqlite3.connect(db_path)

query = """
SELECT id, short_name
FROM raw_entity
WHERE entity_type = 'io_data';
"""
df = pd.read_sql_query(query, conn)

conn.close()

csv_path = r'io_data_entities.csv'
df.to_csv(csv_path, index=False)

conn = sqlite3.connect(db_path)

query = """
SELECT id, short_name
FROM raw_entity
WHERE entity_type = 'functional_step';
"""
df = pd.read_sql_query(query, conn)

conn.close()

csv_path = r'functional_step_entities.csv'
df.to_csv(csv_path, index=False)


In [ ]:
## For io-data entity

import os
import openai
import pandas as pd


# Get environment variable OPENAI_API_KEY
openai.api_key = "YOUR KEY"

PROMPT = '''
Suppose you are a geospatial scientist, given is a sentence:

{}

You are asked to (1) give a short name for this sentence (less than 10 words); 
(2) If there is a specific time scope of spatial location in the string, make them universal; 
(3) Based on your geospatial research knowledge, expand the abbreviation in the string if it has one. 
    For instance, NDVI -> normalized difference vegetation index.
(4) If it is not necessary to revise, keep the original sentence.

Just return the new short name for the sentence exactly without any other words.
'''

def get_completion(prompt, model="gpt-4o-mini", temperature=0.2, messages=None):
    '''
    Retrieve completions from the OpenAI API, preserving conversation history.
    '''
    if messages is None:
        messages = []

    messages.append({"role": "user", "content": prompt})
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    response_content = response.choices[0].message.content
    messages.append({"role": "system", "content": response_content})
    return response_content, messages

# Define file paths
input_csv_path = r'io_data_entities.csv'
output_csv_path = r'io_data_entities_with_content.csv'

# Read input CSV file
df = pd.read_csv(input_csv_path)

# Create new file with column names if it does not exist
if not os.path.exists(output_csv_path):
    pd.DataFrame(columns=['id', 'short_name', 'content']).to_csv(output_csv_path, index=False)

# Read existing data from the output CSV file
df_output = pd.read_csv(output_csv_path)

# Process each short_name
for _, row in df.iterrows():
    cur_id = row['id']
    short_name = row['short_name']
    
    # Create current prompt
    cur_prompt = PROMPT.format(short_name)
    
    # Get generated content
    content, _ = get_completion(cur_prompt, model="gpt-4o-mini")
    
    # Check if id already exists in the output file
    if cur_id in df_output['id'].values:
        # Update existing row's content
        df_output.loc[df_output['id'] == cur_id, 'content'] = content
    else:
        # Add new row
        new_row = {'id': cur_id, 'short_name': short_name, 'content': content}
        df_output = df_output.append(new_row, ignore_index=True)
    
    # Append updates to the output file
    df_output.to_csv(output_csv_path, index=False)
    print(f"New content of {cur_id}: {content}, original content: {short_name}")

print(f"Data has been successfully updated and saved to {output_csv_path}")


In [ ]:
import os
import openai
import pandas as pd

# Get environment variable OPENAI_API_KEY
openai.api_key = "YOUR KEY"input_csv = r'path_to_your_input_csv'

PROMPT = '''
Suppose you are a geospatial scientist, given is a sentence:

{}

You are asked to (1) give a short name for this sentence (less than 10 words); 
(2) If there is a specific time scope of spatial location in the string, make them universal; 
(3) Based on your geospatial research knowledge, expand the abbreviation in the string if it has one. 
    For instance, NDVI -> normalized difference vegetation index.
(4) If it is not necessary to revise, keep the original sentence.

Just return the new short name for the sentence exactly without any other words.
'''

def get_completion(prompt, model="gpt-4o-mini", temperature=0.2, messages=None):
    '''
    Retrieve completions from the OpenAI API, preserving conversation history.
    '''
    if messages is None:
        messages = []

    messages.append({"role": "user", "content": prompt})
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    response_content = response.choices[0].message.content
    messages.append({"role": "system", "content": response_content})
    return response_content, messages

# Define file paths
input_csv_path = r'functional_step_entity.csv'
output_csv_path = r'functional_step_entity_with_content.csv'

# Read input CSV file
df = pd.read_csv(input_csv_path)

# Create new file with column names if it does not exist
if not os.path.exists(output_csv_path):
    pd.DataFrame(columns=['id', 'short_name', 'content']).to_csv(output_csv_path, index=False)

# Read existing data from the output CSV file
df_output = pd.read_csv(output_csv_path)

# Process each short_name
for _, row in df.iterrows():
    cur_id = row['id']
    short_name = row['short_name']
    
    # Create current prompt
    cur_prompt = PROMPT.format(short_name)
    
    # Get generated content
    content, _ = get_completion(cur_prompt, model="gpt-4o-mini")
    
    # Check if id already exists in the output file
    if cur_id in df_output['id'].values:
        # Update existing row's content
        df_output.loc[df_output['id'] == cur_id, 'content'] = content
    else:
        # Add new row
        new_row = {'id': cur_id, 'short_name': short_name, 'content': content}
        df_output = df_output.append(new_row, ignore_index=True)
    
    # Append updates to the output file
    df_output.to_csv(output_csv_path, index=False)
    print(f"New content of {cur_id}: {content}, original content: {short_name}")

print(f"Data has been successfully updated and saved to {output_csv_path}")
